# Домашнее задание к теме Базовые понятия статистики

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики.

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

7. Информация об атрибутах (ключевые параметры):

  4:  rectal temperature — Ректальная температура  (числовой)  
          - линейная  
          - в градусах Цельсия  
          - повышенная температура возможна при инфекции  
          - может снижаться при поздней стадии шока  
          - нормальная температура — 37.8  
          - этот параметр обычно меняется по мере развития проблемы  
               например: может начинаться нормальной, затем повышаться из-за  
                   поражения, снова становиться нормальной, когда лошадь впадает в шок  

  5:  pulse — Пульс  (числовой)  
          - линейный  
          - частота сердечных сокращений в ударах в минуту  
          - отражает состояние сердца: 30–40 — норма для взрослых  
          - редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
          - при болезненных поражениях или шоке пульс обычно повышен  

  9:  mucous membranes — Слизистые оболочки  (категорийный)  
          - субъективная оценка окраски  
          - возможные значения:  
               1 = нормально-розовые  
               2 = ярко-розовые  
               3 = бледно-розовые  
               4 = бледно-цианотичные  
               5 = ярко-красные / воспалённые  
               6 = тёмно-цианотичные  
          - 1 и 2 — нормальное или слегка усиленное кровообращение  
          - 3 — возможно на ранней стадии шока  
          - 4 и 6 — признак серьёзного нарушения циркуляции  
          - 5 — указывает на септицемию  

 11: pain — Боль  (категорийный)  
          - субъективная оценка уровня боли  
          - возможные значения:  
               1 = бодрая, без боли  
               2 = подавленная  
               3 = перемежающаяся лёгкая боль  
               4 = перемежающаяся сильная боль  
               5 = постоянная сильная боль  
          - не следует рассматривать как упорядоченную переменную  
          - чем сильнее боль, тем выше вероятность необходимости операции  
          - обезболивание может частично скрыть уровень боли  

 12: peristalsis — Перистальтика  (категорийный)  
          - показатель активности кишечника; при вздутии или токсичности снижается  
          - возможные значения:  
               1 = гипермоторика  
               2 = нормальная  
               3 = гипомоторика  
               4 = отсутствует  

 13: abdominal distension — Вздутие живота  (категорийный)  
          - ВАЖНЫЙ параметр  
          - возможные значения:  
               1 = нет  
               2 = лёгкое  
               3 = умеренное  
               4 = сильное  
          - сильное вздутие часто требует хирургического вмешательства  

 15: nasogastric reflux — Назогастральный рефлюкс  (категорийный / количественный)  
          - возможные значения:  
               1 = нет  
               2 = > 1 литра  
               3 = < 1 литра  
          - большой объём рефлюкса — признак обструкции кишечника  

 19: packed cell volume — Гематокрит  (числовой)  
          - линейный  
          - процент эритроцитов в крови  
          - норма: 30–50  
          - уровень повышается при обезвоживании или нарушении кровообращения

In [ ]:
import pandas as pd
import numpy as np

In [221]:
attr = [
     {
     'id': 1,
     'label': 'surgery?',
     'label_ru': 'Операция',
     'description': """
1 = Да, была операция  
2 = Нет, лечили без операции
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 2,
     'label': 'Age',
     'label_ru': 'Возраст',
     'description': """
1 = Взрослая лошадь  
2 = Молодая (< 6 месяцев)
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 3,
     'label': 'Hospital Number',
     'label_ru': 'Номер госпитализации',
     'description': """
- числовой идентификатор  
- номер случая, присвоенный лошади  
  (может не быть уникальным, если лошадь лечилась более одного раза)
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 4,
     'label': 'rectal temperature',
     'label_ru': 'Ректальная температура',
     'description': """
- линейная  
- в градусах Цельсия  
- повышенная температура возможна при инфекции  
- может снижаться при поздней стадии шока  
- нормальная температура — 37.8  
- этот параметр обычно меняется по мере развития проблемы  
     например: может начинаться нормальной, затем повышаться из-за  
         поражения, снова становиться нормальной, когда лошадь впадает в шок
""",
     'basis': True,
     'type': 'numerical'
     },
    {
     'id': 5,
     'label': 'pulse',
     'label_ru': 'Пульс',
     'description': """
- линейный  
- частота сердечных сокращений в ударах в минуту  
- отражает состояние сердца: 30–40 — норма для взрослых  
- редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
- при болезненных поражениях или шоке пульс обычно повышен
""",
     'basis': True,
     'type': 'numerical'
     },
    {
     'id': 6,
     'label': 'respiratory rate',
     'label_ru': 'Частота дыхания',
     'description': """
- линейная  
- нормальная частота — 8–10  
- полезность сомнительна из-за больших колебаний
""",
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 7,
     'label': 'temperature of extremities',
     'label_ru': 'Температура конечностей',
     'description': """
- субъективный показатель периферического кровообращения  
- возможные значения:  
     1 = Нормальные  
     2 = Тёплые  
     3 = Прохладные  
     4 = Холодные  
- прохладные или холодные конечности указывают на возможный шок  
- горячие конечности коррелируют с повышенной ректальной температурой
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 8,
     'label': 'peripheral pulse',
     'label_ru': 'Периферический пульс',
     'description': """
- субъективный показатель  
- возможные значения:  
     1 = нормальный  
     2 = повышенный  
     3 = пониженный  
     4 = отсутствует  
- нормальный или повышенный — признак хорошего кровообращения,  
  пониженный или отсутствующий — плохой перфузии
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 9,
     'label': 'mucous membranes',
     'label_ru': 'Слизистые оболочки',
     'description': """
- субъективная оценка окраски  
- возможные значения:  
     1 = нормально-розовые  
     2 = ярко-розовые  
     3 = бледно-розовые  
     4 = бледно-цианотичные  
     5 = ярко-красные / воспалённые  
     6 = тёмно-цианотичные  
- 1 и 2 — нормальное или слегка усиленное кровообращение  
- 3 — возможно на ранней стадии шока  
- 4 и 6 — признак серьёзного нарушения циркуляции  
- 5 — указывает на септицемию
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 10,
     'label': 'capillary refill time',
     'label_ru': 'Время капиллярного наполнения',
     'description': """
- клиническая оценка: чем дольше наполнение, тем хуже циркуляция  
- возможные значения:  
     1 = < 3 секунд  
     2 = ≥ 3 секунд
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 11,
     'label': 'pain',
     'label_ru': 'Боль',
     'description': """
- субъективная оценка уровня боли  
- возможные значения:  
     1 = бодрая, без боли  
     2 = подавленная  
     3 = перемежающаяся лёгкая боль  
     4 = перемежающаяся сильная боль  
     5 = постоянная сильная боль  
- не следует рассматривать как упорядоченную переменную  
- чем сильнее боль, тем выше вероятность необходимости операции  
- обезболивание может частично скрыть уровень боли
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 12,
     'label': 'peristalsis',
     'label_ru': 'Перистальтика',
     'description': """
- показатель активности кишечника; при вздутии или токсичности снижается  
- возможные значения:  
     1 = гипермоторика  
     2 = нормальная  
     3 = гипомоторика  
     4 = отсутствует
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 13,
     'label': 'abdominal distension',
     'label_ru': 'Вздутие живота',
     'description': """
- ВАЖНЫЙ параметр  
- возможные значения:  
     1 = нет  
     2 = лёгкое  
     3 = умеренное  
     4 = сильное  
- сильное вздутие часто требует хирургического вмешательства
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 14,
     'label': 'nasogastric tube',
     'label_ru': 'Назогастральная трубка',
     'description': """
- наличие газа, выходящего из трубки  
- возможные значения:  
     1 = нет  
     2 = немного  
     3 = значительное количество  
- большое количество газа вызывает дискомфорт
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 15,
     'label': 'nasogastric reflux',
     'label_ru': 'Назогастральный рефлюкс',
     'description': """
- возможные значения:  
     1 = нет  
     2 = > 1 литра  
     3 = < 1 литра  
- большой объём рефлюкса — признак обструкции кишечника
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 16,
     'label': 'nasogastric reflux pH',
     'label_ru': 'pH назогастрального рефлюкса',
     'description': """
- линейный  
- шкала 0–14, 7 — нейтрально  
- нормальные значения: 3–4
""",
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 17,
     'label': 'rectal examination - feces',
     'label_ru': 'Ректальный осмотр (кал)',
     'description': """
- возможные значения:  
     1 = нормальный  
     2 = увеличенный  
     3 = уменьшенный  
     4 = отсутствует  
- отсутствие кала, вероятно, указывает на непроходимость
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 18,
     'label': 'abdomen',
     'label_ru': 'Живот',
     'description': """
- возможные значения:  
     1 = нормальный  
     2 = другое  
     3 = плотный кал в толстом кишечнике  
     4 = вздутие тонкого кишечника  
     5 = вздутие толстого кишечника  
- 3 — вероятно механическая непроходимость (лечится медикаментозно)  
- 4 и 5 — хирургические поражения
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 19,
     'label': 'packed cell volume',
     'label_ru': 'Гематокрит',
     'description': """
- линейный  
- процент эритроцитов в крови  
- норма: 30–50  
- уровень повышается при обезвоживании или нарушении кровообращения
""",
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 20,
     'label': 'total protein',
     'label_ru': 'Общий белок',
     'description': """
- линейный  
- норма: 6–7.5 г/дл  
- повышенные значения — признак обезвоживания
""",
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 21,
     'label': 'abdominocentesis appearance',
     'label_ru': 'Внешний вид жидкости при абдоминоцентезе',
     'description': """
- берётся жидкость из брюшной полости  
- возможные значения:  
     1 = прозрачная  
     2 = мутная  
     3 = серозно-кровянистая  
- мутная или серозно-кровянистая жидкость указывает на повреждение кишечника
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 22,
     'label': 'abdominocentesis total protein',
     'label_ru': 'Белок при абдоминоцентезе',
     'description': """
- линейный  
- чем выше уровень белка, тем выше вероятность повреждения кишечника (г/дл)
""",
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 23,
     'label': 'outcome',
     'label_ru': 'Исход',
     'description': """
- что в итоге произошло с лошадью?  
- возможные значения:  
     1 = выжила  
     2 = умерла  
     3 = эвтаназирована
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 24,
     'label': 'surgical lesion?',
     'label_ru': 'Хирургическое поражение?',
     'description': """
- ретроспективно: была ли проблема хирургической?  
- все случаи либо оперировались, либо вскрывались  
- возможные значения:  
     1 = Да  
     2 = Нет
""",
     'basis': True,
     'type': 'categorical'
     },
    {
     'id': 25,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',
     'description': """
- первая цифра — место поражения:  
     1 = желудок  
     2 = тонкий кишечник  
     3 = толстая кишка  
     4 = толстая кишка и слепая  
     5 = слепая кишка  
     6 = поперечная ободочная  
     7 = прямая/нисходящая кишка  
     8 = матка  
     9 = мочевой пузырь  
     11 = все отделы кишечника  
     00 = нет  
- вторая цифра — тип:  
     1 = простое  
     2 = странгуляция  
     3 = воспаление  
     4 = другое  
- третья цифра — подтип:  
     1 = механическое  
     2 = паралитическое  
     0 = н/д  
- четвёртая цифра — код:  
     1 = закупорка  
     2 = внутреннее поражение  
     3 = внешнее поражение  
     4 = адинамическое  
     5 = заворот/торсия  
     6 = инвагинация  
     7 = тромбоэмболия  
     8 = грыжа  
     9 = липома / инкарцерация селезёнки  
     10 = смещение  
     0 = н/д
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 26,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',
     'description': """
- первая цифра — место поражения:  
     1 = желудок  
     2 = тонкий кишечник  
     3 = толстая кишка  
     4 = толстая кишка и слепая  
     5 = слепая кишка  
     6 = поперечная ободочная  
     7 = прямая/нисходящая кишка  
     8 = матка  
     9 = мочевой пузырь  
     11 = все отделы кишечника  
     00 = нет  
- вторая цифра — тип:  
     1 = простое  
     2 = странгуляция  
     3 = воспаление  
     4 = другое  
- третья цифра — подтип:  
     1 = механическое  
     2 = паралитическое  
     0 = н/д  
- четвёртая цифра — код:  
     1 = закупорка  
     2 = внутреннее поражение  
     3 = внешнее поражение  
     4 = адинамическое  
     5 = заворот/торсия  
     6 = инвагинация  
     7 = тромбоэмболия  
     8 = грыжа  
     9 = липома / инкарцерация селезёнки  
     10 = смещение  
     0 = н/д
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 27,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',
     'description': """
- первая цифра — место поражения:  
     1 = желудок  
     2 = тонкий кишечник  
     3 = толстая кишка  
     4 = толстая кишка и слепая  
     5 = слепая кишка  
     6 = поперечная ободочная  
     7 = прямая/нисходящая кишка  
     8 = матка  
     9 = мочевой пузырь  
     11 = все отделы кишечника  
     00 = нет  
- вторая цифра — тип:  
     1 = простое  
     2 = странгуляция  
     3 = воспаление  
     4 = другое  
- третья цифра — подтип:  
     1 = механическое  
     2 = паралитическое  
     0 = н/д  
- четвёртая цифра — код:  
     1 = закупорка  
     2 = внутреннее поражение  
     3 = внешнее поражение  
     4 = адинамическое  
     5 = заворот/торсия  
     6 = инвагинация  
     7 = тромбоэмболия  
     8 = грыжа  
     9 = липома / инкарцерация селезёнки  
     10 = смещение  
     0 = н/д
""",
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 28,
     'label': 'cp_data',
     'label_ru': 'Наличие патологоанатомических данных',
     'description': """
- есть ли патологоанатомические данные?  
     1 = Да  
     2 = Нет  
- переменная не имеет значения, так как данные не собраны
""",
     'basis': False,
     'type': 'categorical'
     }
]

In [225]:
def prepare_data(file, attr, basis = True):
     df = pd.read_csv(file, header=None)
     prepared_df = pd.DataFrame()
     for element in attr:
          if (basis == True and element['basis'] == True) or (basis == False):
               if element['type'] == 'numerical':
                    prepared_df[element['id'] - 1] = pd.to_numeric(df[element['id'] - 1], errors='coerce')
               else:
                    prepared_df[element['id'] - 1] = df[element['id'] - 1].astype(str)
     
               # prepared_df = prepared_df.rename(columns={element['id'] - 1: element['label_ru']})
               # Обязательно переименовывать! Если записывать сразу в название, одинаковые названия перезапишутся
     # display(prepared_df)
     return prepared_df

In [226]:
def describe_data(df):
     data = pd.DataFrame()
     # display(df)
     for column in df:
          # print('column')
          # display(column)
          # print('data')
          # display(data)
          # print('add')
          described_column = pd.DataFrame(df[column].describe())
          # described_column = described_column
          data = pd.concat([data,described_column], axis=1)
          # data = pd.DataFrame(df[column].describe())
          # display(data)
          
          # data[column] = described_column[0]
          # display('suka', suka)
     return data

In [ ]:
horses = prepare_data('horse_data.csv', attr, False)
display(horses)
described_horses = describe_data(horses)
display(described_horses)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2,1,530101,38.5,66.0,28.0,3,3,?,2,...,45.0,8.4,?,NaN,2,2,11300,0,0,2
1,1,1,534817,39.2,88.0,20.0,?,?,4,1,...,50.0,85.0,2,2.0,3,2,2208,0,0,2
2,2,1,530334,38.3,40.0,24.0,1,1,3,1,...,33.0,6.7,?,NaN,1,2,0,0,0,1
3,1,9,5290409,39.1,164.0,84.0,4,1,6,2,...,48.0,7.2,3,5.3,2,1,2208,0,0,1
4,2,1,530255,37.3,104.0,35.0,?,?,6,2,...,74.0,7.4,?,NaN,2,2,4300,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,533886,NaN,120.0,70.0,4,?,4,2,...,55.0,65.0,?,NaN,3,2,3205,0,0,2
296,2,1,527702,37.2,72.0,24.0,3,2,4,2,...,44.0,NaN,3,3.3,3,1,2208,0,0,1
297,1,1,529386,37.5,72.0,30.0,4,3,4,1,...,60.0,6.8,?,NaN,2,1,3205,0,0,2
298,1,1,530612,36.5,100.0,24.0,3,3,3,1,...,50.0,6.0,3,3.4,1,1,2208,0,0,1


object


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,300,300,300,240.000000,276.000000,242.000000,300,300,300,300,...,271.000000,267.000000,300,102.000000,300,300,300,300,300,300
unique,3,2,284,NaN,NaN,NaN,5,5,7,4,...,NaN,NaN,4,NaN,4,2,61,6,2,2
top,1,1,532349,NaN,NaN,NaN,3,1,1,1,...,NaN,NaN,?,NaN,1,1,0,0,0,2
freq,180,276,2,NaN,NaN,NaN,109,115,79,188,...,NaN,NaN,165,NaN,178,191,56,293,299,201
mean,NaN,NaN,NaN,38.167917,71.913043,30.417355,NaN,NaN,NaN,NaN,...,46.295203,24.456929,NaN,3.019608,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,0.732289,28.630557,17.642231,NaN,NaN,NaN,NaN,...,10.419335,27.475009,NaN,1.968567,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,35.400000,30.000000,8.000000,NaN,NaN,NaN,NaN,...,23.000000,3.300000,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,37.800000,48.000000,18.500000,NaN,NaN,NaN,NaN,...,38.000000,6.500000,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,38.200000,64.000000,24.500000,NaN,NaN,NaN,NaN,...,45.000000,7.500000,NaN,2.250000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,38.500000,88.000000,36.000000,NaN,NaN,NaN,NaN,...,52.000000,57.000000,NaN,3.900000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# attributes_ru = [
#     'Операция',
#     'Возраст',
#     'Номер госпитализации',
#     'Ректальная температура',
#     'Пульс',
#     'Частота дыхания',
#     'Температура конечностей',
#     'Периферический пульс',
#     'Слизистые оболочки',
#     'Время капиллярного наполнения',
#     'Боль',
#     'Перистальтика',
#     'Вздутие живота',
#     'Назогастральная трубка',
#     'Назогастральный рефлюкс',
#     'pH назогастрального рефлюкса',
#     'Ректальный осмотр — кал',
#     'Живот',
#     'Гематокрит',
#     'Общий белок',
#     'Внешний вид жидкости при абдоминоцентезе',
#     'Белок при абдоминоцентезе',
#     'Исход',
#     'Хирургическое поражение',
#     'Тип поражения (25)',
#     'Тип поражения (26)',
#     'Тип поражения (27)',
#     'Наличие патологоанатомических данных'
# ]





# Прочитать данные
# def read_data(file):
#     data = pd.read_csv(file, header=None)
#     return data

# Добавить данным названия столбцов


     # return df
# Составить датафрейм с результатами 


# horses = pd.read_csv("horse_data.csv")

# horses = read_data("horse_data.csv")
# horses = prepare_data('horse_data.csv', attr, False)
# described_horses = describe_data(horses)
# display(described_horses)




# horses.columns = attributes_ru

# horses_important = horses[[
#     'Ректальная температура',      # числовой, отражает инфекцию, шок
#     'Пульс',                       # числовой, повышается при боли и шоке
#     'Слизистые оболочки',          # категориальный, указывает на степень шока и перфузию тканей
#     'Боль',                        # категориальный, основной симптом тяжести состояния
#     'Перистальтика',               # категориальный, снижение указывает на кишечную непроходимость
#     'Вздутие живота',              # категориальный, важнейший показатель, часто требует операции
#     'Назогастральный рефлюкс',     # категориальный/количественный, большой объём — признак обструкции
#     'Гематокрит'                   # числовой, повышен при обезвоживании и шоке
# ]]

# def separate_print(value, description=''):
#     print(value,
#           '\n',
#           description,
#           '\n',
#           '---------------------',
#           '\n')



# # Количественные
# horses_rectal_temperature = pd.to_numeric(horses_important['Ректальная температура'], errors='coerce').describe()
# separate_print(horses_rectal_temperature,
#  """
# - линейная  
# - в градусах Цельсия  
# - повышенная температура возможна при инфекции  
# - может снижаться при поздней стадии шока  
# - нормальная температура — 37.8  
# - этот параметр обычно меняется по мере развития проблемы  
#      например: может начинаться нормальной, затем повышаться из-за  
#          поражения, снова становиться нормальной, когда лошадь впадает в шок
# """)

# horses_pulse = pd.to_numeric(horses_important['Пульс'], errors='coerce').describe()
# separate_print(horses_pulse,
# """
# - линейный  
# - частота сердечных сокращений в ударах в минуту  
# - отражает состояние сердца: 30–40 — норма для взрослых  
# - редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
# - при болезненных поражениях или шоке пульс обычно повышен  
# """)

# horses_packed_cell_volume = pd.to_numeric(horses_important['Гематокрит'], errors='coerce').describe()
# separate_print(horses_packed_cell_volume,
# """
# - линейный  
# - процент эритроцитов в крови  
# - норма: 30–50  
# - уровень повышается при обезвоживании или нарушении кровообращения
# """)

# # Категориальные
# horses_mucous_membranes = horses_important['Слизистые оболочки'].describe()
# separate_print(horses_mucous_membranes,
# """
# - субъективная оценка окраски  
# - возможные значения:  
#      1 = нормально-розовые  
#      2 = ярко-розовые  
#      3 = бледно-розовые  
#      4 = бледно-цианотичные  
#      5 = ярко-красные / воспалённые  
#      6 = тёмно-цианотичные  
# - 1 и 2 — нормальное или слегка усиленное кровообращение  
# - 3 — возможно на ранней стадии шока  
# - 4 и 6 — признак серьёзного нарушения циркуляции  
# - 5 — указывает на септицемию  
# """)

# horses_pain = horses_important['Боль'].describe()
# separate_print(horses_pain,
# """
# - субъективная оценка уровня боли  
# - возможные значения:  
#      1 = бодрая, без боли  
#      2 = подавленная  
#      3 = перемежающаяся лёгкая боль  
#      4 = перемежающаяся сильная боль  
#      5 = постоянная сильная боль  
# - не следует рассматривать как упорядоченную переменную  
# - чем сильнее боль, тем выше вероятность необходимости операции  
# - обезболивание может частично скрыть уровень боли  
# """
# )

# horses_peristalsis = horses_important['Перистальтика'].describe()
# separate_print(horses_peristalsis,
# """
# - показатель активности кишечника; при вздутии или токсичности снижается  
# - возможные значения:  
#      1 = гипермоторика  
#      2 = нормальная  
#      3 = гипомоторика  
#      4 = отсутствует  
# """)

# horses_abdominal_distension = horses_important['Вздутие живота'].describe()
# separate_print(horses_abdominal_distension,
# """
# - ВАЖНЫЙ параметр  
# - возможные значения:  
#      1 = нет  
#      2 = лёгкое  
#      3 = умеренное  
#      4 = сильное  
# - сильное вздутие часто требует хирургического вмешательства  
# """)

# horses_nasogastric_reflux = horses_important['Назогастральный рефлюкс'].describe()
# separate_print(horses_nasogastric_reflux,
# """
# - возможные значения:  
#      1 = нет  
#      2 = > 1 литра  
#      3 = < 1 литра  
# - большой объём рефлюкса — признак обструкции кишечника  
# """)






# print(base_metrics)

# display(horses_rectal_temperature.head())

,Операция,Возраст,Номер госпитализации,Ректальная температура,Пульс,Частота дыхания,Температура конечностей,Периферический пульс,Слизистые оболочки,Время капиллярного наполнения,...,Гематокрит,Общий белок,Внешний вид жидкости при абдоминоцентезе,Белок при абдоминоцентезе,Исход,Хирургическое поражение?,Тип поражения,Тип поражения,Тип поражения,Наличие патологоанатомических данных
0,2,1,530101,38.5,66.0,28.0,3,3,?,2,...,45.0,8.4,?,NaN,2,2,11300,0,0,2
1,1,1,534817,39.2,88.0,20.0,?,?,4,1,...,50.0,85.0,2,2.0,3,2,2208,0,0,2
2,2,1,530334,38.3,40.0,24.0,1,1,3,1,...,33.0,6.7,?,NaN,1,2,0,0,0,1
3,1,9,5290409,39.1,164.0,84.0,4,1,6,2,...,48.0,7.2,3,5.3,2,1,2208,0,0,1
4,2,1,530255,37.3,104.0,35.0,?,?,6,2,...,74.0,7.4,?,NaN,2,2,4300,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,533886,NaN,120.0,70.0,4,?,4,2,...,55.0,65.0,?,NaN,3,2,3205,0,0,2
296,2,1,527702,37.2,72.0,24.0,3,2,4,2,...,44.0,NaN,3,3.3,3,1,2208,0,0,1
297,1,1,529386,37.5,72.0,30.0,4,3,4,1,...,60.0,6.8,?,NaN,2,1,3205,0,0,2
298,1,1,530612,36.5,100.0,24.0,3,3,3,1,...,50.0,6.0,3,3.4,1,1,2208,0,0,1


count     300
unique      3
top         1
freq      180
Name: Операция, dtype: object

count     300.0
unique      NaN
top         NaN
freq        NaN
Name: Возраст, dtype: float64

count     300.0
unique      NaN
top         NaN
freq        NaN
Name: Номер госпитализации, dtype: float64

count     240.0
unique      NaN
top         NaN
freq        NaN
Name: Ректальная температура, dtype: float64

...

count     300
unique      4
top         ?
freq      165
Name: Внешний вид жидкости при абдоминоцентезе, dtype: object

count     102.0
unique      NaN
top         NaN
freq        NaN
Name: Белок при абдоминоцентезе, dtype: float64

count     300
unique      4
top         1
freq      178
Name: Исход, dtype: object

count     300.0
unique      NaN
top         NaN
freq        NaN
Name: Хирургическое поражение?, dtype: float64

ValueError: Cannot set a DataFrame with multiple columns to the single column Тип поражения